#### 한국어 코퍼스 전처리
- DataSet : Korpora에서 Loading
- 형태소 분석기 설정 : 본인 원하는 것
- 단어사전 생성

[1] Module Loading 및 Data 준비 <hr>

In [8]:
# Module Loading
from Korpora import Korpora                             # Open Korean DataSet
from konlpy.tag import *                                # 형태소 분석기
import spacy                                            # 형태소 분석기

from torchtext.data.utils import get_tokenizer          # Pytorch 자연어처리 Module
from torchtext.vocab import *                           # Pytorch 자연어처리 단어사전 관련 Module
from torch.utils.data import Dataset, DataLoader        # Pytorch DataSet 관련 Module

import pandas as pd
import numpy as np

c:\Users\hwans\anaconda3\envs\NLP_018_230_38\lib\site-packages\torchtext\data\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\hwans\anaconda3\envs\NLP_018_230_38\lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\hwans\anaconda3\envs\NLP_018_230_38\lib\site-packages\torchtext\utils.py:4: UserWarning: 
/!\ 

In [9]:
# Data Loading
nsmc = Korpora.load("nsmc")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\hwans\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\hwa

In [10]:
nsmc.train[0], nsmc.test[0]

(LabeledSentence(text='아 더빙.. 진짜 짜증나네요 목소리', label=0),
 LabeledSentence(text='굳 ㅋ', label=1))

In [11]:
# Test DataSet을 DataFrame으로 로딩
nsmcDF = pd.DataFrame(nsmc.test)
nsmcDF.info()

# text가 Frature가 되고 label이 Target

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50000 non-null  object
 1   label   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [12]:
# 원래 tarin data를 가져와서 해야하지만 train data가 많아서 test data로 진행하는 것

- [2] 커스텀 DataSet 정의 <hr>

In [13]:
# 단어사전을 생성 시 활용
# 단어사전을 생성하는거라서 tensor 값이 아닌 텍스트를 줘야함 

class TextDataset(Dataset) :
    def __init__ (self, feature, label) :
        self.feature = feature
        self.label = label
        self.length = feature.shape[0]

    def __len__(self) :
        return self.length

    def __getitem__ (self, index) :
        return self.feature.iloc[index], self.label.iloc[index]

In [14]:
# nsmc의 DataSet 인스턴스 생성
nsmcDS = TextDataset(nsmcDF["text"], nsmcDF["label"])

In [15]:
# Data 확인
for f, l in nsmcDS :
    print(f, l)
    break

굳 ㅋ 1


[3] Text Data 전처리 <hr>
- 토큰화/정제 (불용어, 구두점, 오타, ...)
- 단어사전 생성
- 모든 문장을 수치화 
- -------------------

In [16]:
# 불용어 제거 함수
from nltk.corpus import stopwords

# 토큰화
from nltk.tokenize import word_tokenize

In [31]:
# Tokenize 생성
get_tokenizer(Okt().morphs)            # 그냥 함수를 호출하는 것 -> 저장을 해야함     
# morphs: 형태소를 분석해주는 애

def preprocessing(text) :
        # 불용어 처리
        kor_sw = stopwords.words(text)
        text = text.lower()
        tokens = word_tokenize(text)
        for token in tokens :
                if token not in kor_sw :
                        tokens.remove(token)

okt_tokenizer = get_tokenizer(Okt().morphs)  
spacy_tokenizer = get_tokenizer("spacy", language="ko_core_news_sm")        # language에 내가 설치했던 이름을 넣어줘야함

In [32]:
# Text DataSet에서 처리할 DataSet만 Loading해서 토큰화 진행하는 반복자 함수
def generateToken(dataset) :
    for text, label in dataset :
        # 토큰화 
        yield okt_tokenizer(text)
        pass

In [33]:
# 단어사전 생성 
UNK_TOKEN = "<UNK>"
PAD_TOKEN = "<PAD>"

nsmcVocab = build_vocab_from_iterator(iterator=generateToken(nsmcDS),
                                      specials=[UNK_TOKEN])

nsmcVocab.set_default_index(nsmcVocab[UNK_TOKEN])                   # OOV 토큰의 인덱스를 지정할 수 있음

In [34]:
len(nsmcVocab)

56812